---

In [121]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re

In [122]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [123]:
colnames=['ID', 'Social network', 'Sentiment', 'Text'] 
df_train = pd.read_csv('twitter_training.csv', names=colnames, header=None)
df_val = pd.read_csv('twitter_validation.csv', names=colnames, header=None)



df_train.info()
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              74682 non-null  int64 
 1   Social network  74682 non-null  object
 2   Sentiment       74682 non-null  object
 3   Text            73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              1000 non-null   int64 
 1   Social network  1000 non-null   object
 2   Sentiment       1000 non-null   object
 3   Text            1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [124]:

df_val

,ID,Social network,Sentiment,Text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...
...,...,...,...,...
995,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
996,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [125]:
df_val = df_val.replace(to_replace='None', value=np.nan).dropna()
df_train= df_train.replace(to_replace='None', value=np.nan).dropna()

In [126]:
STOPWORDS  = stopwords.words('english') + ['im']
STOPWORDS.remove('not')
STOPWORDS.remove('no')

lemmatizer = WordNetLemmatizer()

#### Функция очистки и лемматизации текста
def clean_text(text):
  #очистить текст от тегов html и привести к нижнему регистру
  text = BeautifulSoup(text).get_text().lower()

  #очистить текст от ссылок html
  #text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)

  #очистить текст от всех символов кроме букв латинского алфавита
  text = re.sub("[^a-z]", " ", text)

  #разбить текст на слова
  #text_words = text.split()

  #выполнить лематизацию текста и удалить стоп-слова
  #result_words = [lemmatizer.lemmatize(word) for word in text_words if not word in STOPWORDS]
    
  return  text

In [127]:
df_train['Text'] = df_train['Text'].apply(lambda x: clean_text(x))
df_val['Text'] = df_val['Text'].apply(lambda x: clean_text(x))

C:\Users\anon\AppData\Local\Temp\ipykernel_16644\519989603.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text).get_text().lower()


In [128]:
df_train['Text']

0        im getting on borderlands and i will murder yo...
1        i am coming to the borders and i will kill you...
2        im getting on borderlands and i will kill you ...
3        im coming on borderlands and i will murder you...
4        im getting on borderlands   and i will murder ...
                               ...                        
74677    just realized that the windows partition of my...
74678    just realized that my mac window partition is ...
74679    just realized the windows partition of my mac ...
74680    just realized between the windows partition of...
74681    just like the windows partition of my mac is l...
Name: Text, Length: 73996, dtype: object

In [129]:
from sklearn.preprocessing import LabelEncoder
labeles = LabelEncoder()
labels = labeles.fit_transform(df_train['Sentiment'])

In [130]:
labeles.classes_

array(['Irrelevant', 'Negative', 'Neutral', 'Positive'], dtype=object)

---

In [131]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    DataCollatorWithPadding,
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    pipeline
)
import numpy as np

In [132]:
BATCH_SIZE = 32
NUM_PROCS = 32
LR = 0.00005
EPOCHS = 10

In [133]:
id2label = {
   0:'Irrelevant', 1:'Negative', 2:'Neutral', 3:'Positive'
}
label2id = {
   'Irrelevant':0, 'Negative':1, 'Neutral':2, 'Positive':3
}

In [134]:
from sklearn.model_selection import train_test_split

In [135]:
#X_train, X_test, y_train, y_test = train_test_split(queries, y, test_size=0.2, random_state=42)

In [136]:
#X_train, X_test, y_train, y_test = train_test_split(df['Message'], labels, test_size=0.2, random_state=42)
X_train=df_train['Text']
X_test=df_val['Text']
y_train=df_train['Sentiment']
y_test=df_val['Sentiment']

In [137]:
# from sklearn.utils.class_weight import compute_class_weight
# classes = np.unique(y)
# class_weights = compute_class_weight('balanced', classes=classes, y=y)

In [138]:
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(labels)
class_weights = compute_class_weight('balanced', classes=classes, y=labels)

In [139]:
import torch
# Преобразуйте веса классов в тензор PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [140]:
import pandas as pd
data = pd.DataFrame({'text': X_train, 'label': y_train})
data_test = pd.DataFrame({'text': X_test, 'label': y_test})

In [141]:
data_test["label"].value_counts()

label
Neutral       285
Positive      277
Negative      266
Irrelevant    172
Name: count, dtype: int64

In [142]:
from datasets import Dataset
dataset_train = Dataset.from_pandas(data)

In [143]:
dataset_test = Dataset.from_pandas(data_test)

In [144]:
from transformers import AutoTokenizer, MobileBertModel

In [145]:
token = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
model = MobileBertModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

You are using a model of type bert to instantiate a model of type mobilebert. This is not supported for all configurations of models and can yield errors.
Some weights of MobileBertModel were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.embedding_transformation.bias', 'embeddings.embedding_transformation.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'enco

In [146]:
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return self.tokenizer(
            examples["text"],
            padding="max_length",
            max_length=128,#128
            truncation=True,
            
        )

In [147]:
tokenizer_wrapper = TokenizerWrapper(token)

In [148]:
tokenized_train = dataset_train.map(
    tokenizer_wrapper.tokenize_function, 
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=2
)

Map (num_proc=2):   0%|          | 0/73996 [00:00<?, ? examples/s]

In [149]:
tokenized_test = dataset_test.map(
    tokenizer_wrapper.tokenize_function, 
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=2
)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [150]:
import evaluate
accuracy = evaluate.load('accuracy')

In [151]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [152]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google/bert_uncased_L-2_H-128_A-2",#"google/mobilebert-uncased",  "google/bert_uncased_L-2_H-128_A-2"
    num_labels=4, 
    id2label=id2label, 
    label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [153]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

4,386,436 total parameters.
4,386,436 training parameters.


In [163]:
batch_size = 16
training_args = TrainingArguments(
    output_dir="bert_uncased_L-2_H-128_A-2",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    
    #fp16=True
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'padding'

In [155]:
import torch.nn.functional as F
class_weights=class_weights.to("cuda")
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = F.cross_entropy(logits, labels, weight=class_weights)
        return (loss, outputs) if return_outputs else loss

In [156]:
data_collator = DataCollatorWithPadding(tokenizer=token)

In [162]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=token,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)

TypeError: Trainer.__init__() got an unexpected keyword argument 'padding'

In [159]:
history = trainer.train()

  0%|          | 0/46250 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
# # Получаем предсказания
predictions = trainer.predict(tokenized_test)

# # Предсказанные классы
predicted_classes = predictions.predictions.argmax(-1)

  0%|          | 0/70 [00:00<?, ?it/s]

In [ ]:
from sklearn import metrics
print("Precision: %f" % metrics.precision_score(data_test["label"], predicted_classes, average='macro'))
print("Recall: %f" % metrics.recall_score(data_test["label"], predicted_classes, average='macro'))
print("F1-Score: %f" % metrics.f1_score(data_test["label"], predicted_classes, average='macro'))

Precision: 0.980633
Recall: 0.980633
F1-Score: 0.980633


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix(data_test["label"], predicted_classes)

array([[961,   5],
       [  5, 144]], dtype=int64)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()

In [ ]:
accelerator.wait_for_everyone()
accelerator.save_model(model, "./out", max_shard_size="1GB", safe_serialization=True)

In [ ]:
!optimum-cli export onnx --model "./bert_uncased_L-2_H-128_A-2/checkpoint-37500"  --task text-classification squad_onnx_128/

"optimum-cli" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
